In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ResNet 기본 블럭 정의

In [2]:
class BasicBlock(nn.Module):
    
    expansion = 1  # 확장 비율 변수 > ResNet block 채널 수를 확장하는 경우에 숫자를 늘려주면 됨 (default = 1)
    
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
                
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                # in_channel과 self.expansion * out_channels 사이의 차이를 보상하기위해 1x1 컨볼루션 진행 
                # >>> 더 깊은 네트워크에서 안정적 학습
            )
        
    def forward(self, x):
        out = F.relu(self.bn1(x))
        residual = out
        
        out = self.conv1(out)
        out = F.relu(self.bn2(out))
        out = self.conv2(out)
        
        if hasattr(self, 'shortcut') :   #  기존 shortcut이 있는지 확인
            residual = self.shortcut(x)
            
        out += residual
        
        return out

# ResNet 모델 정의

In [3]:
class PreActResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(PreActResNet, self).__init__()
        
        self.in_Channels = 64
        
        self.conv1 = nn.Conv2d(3, self.in_Channels, kernel_size=3, stride=2, padding=3, bias=False)
        
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
    def _make_layer(self, block, out_channels, blocks, stride=1): # 내부 매서드 >> ResNet 내에서 반복적으로 사용되는 블록 구성 역할 수행
        
        layers = []
        layers.append(block(self.in_Channels, out_channels, stride=1))
        self.in_Channels = out_channels * block.expansion
        
        for _ in range(1, blocks):
            layers.append(block(self.in_Channels, out_channels))
            
        return nn.Sequential(*layers)  # nn.Sequential >> 모델을 구성하기위한 컨테이너 클래스: 생성된 모든 layer들을 순차적으로 추가해줌
    
    def forward(self, x):
        x = self.conv1(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        
        return x

In [7]:
def preact_resnet34(num_classes = 1000):
    
    return PreActResNet(BasicBlock, [3,4,6,3], num_classes)

def preact_resnet18(num_classes=1000):
    
    return PreActResNet(BasicBlock, [2,2,2,2], num_classes)


model = preact_resnet34(num_classes = 1000)
inputs = torch.randn(1,3,224,224)
output = model(inputs)
print(output.shape)

model = preact_resnet18(num_classes = 1000)
inputs = torch.randn(1,3,224,224)
output = model(inputs)
print(output.shape)

torch.Size([1, 1000])
torch.Size([1, 1000])
